In [ ]:
##### imports ##########

In [1]:
import numpy as np
from dask.distributed import Client, LocalCluster
import dask.dataframe as dd
import pandas as pd

In [2]:
! ls -lrt D:/uber-taxi-demand/data/yellow_tripdata_2022-*.parquet | wc -l

12


In [3]:
! ls D:/uber-taxi-demand/data/yellow_tripdata_2022-*.parquet

D:/uber-taxi-demand/data/yellow_tripdata_2022-01.parquet
D:/uber-taxi-demand/data/yellow_tripdata_2022-02.parquet
D:/uber-taxi-demand/data/yellow_tripdata_2022-03.parquet
D:/uber-taxi-demand/data/yellow_tripdata_2022-04.parquet
D:/uber-taxi-demand/data/yellow_tripdata_2022-05.parquet
D:/uber-taxi-demand/data/yellow_tripdata_2022-06.parquet
D:/uber-taxi-demand/data/yellow_tripdata_2022-07.parquet
D:/uber-taxi-demand/data/yellow_tripdata_2022-08.parquet
D:/uber-taxi-demand/data/yellow_tripdata_2022-09.parquet
D:/uber-taxi-demand/data/yellow_tripdata_2022-10.parquet
D:/uber-taxi-demand/data/yellow_tripdata_2022-11.parquet
D:/uber-taxi-demand/data/yellow_tripdata_2022-12.parquet


In [4]:
##### Create Cluster ##########

In [4]:
cluster = LocalCluster(
    name='timeseries',
    n_workers=2,
    threads_per_worker=1,
    host='localhost',
    scheduler_port=8786,
    asynchronous=True, 
    processes=False
)
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 7.83 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:58296,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 7.83 GiB
Comm: tcp://127.0.0.1:58318,Total threads: 2
Dashboard: http://127.0.0.1:58324/status,Memory: 1.96 GiB
Nanny: tcp://127.0.0.1:58300,


In [8]:
def create_hourly_stamp(row: pd.Timestamp) -> pd.Timestamp:
    """
    It Will create hourly time stamp for Forecasting
    """
    hour = row.time().hour
    date = row.date()
    return pd.Timestamp(hour=hour, minute=0, second=0, microsecond=0, day=date.day, month=date.month, year=date.year)

In [ ]:
############## Steps ####################

In [6]:
def optimeToFitMemory(ddf):
    new_types = dict(
        int32 = ['passenger_count',],
        int16 = ['VendorID',]
    )
    for key in new_types:
        for col in new_types[key]:
            ddf[col] = ddf[col].astype(key)
    return ddf

In [23]:
ddf = dd.read_parquet('D:/uber-taxi-demand/data/yellow_tripdata_2022-01.parquet', npartitions=4)
s1 = ddf.loc[
    ddf.tpep_pickup_datetime>'2021-12-31 23:00:00', 
    ['tpep_pickup_datetime','passenger_count', 'VendorID']
]
s2 = s1.set_index('tpep_pickup_datetime', sorted=True)
s3 = s2.ffill()
s4 = s3.resample('H').agg({'passenger_count': 'sum', 'VendorID': 'count'})
s5 = s4.map_partitions(optimeToFitMemory)
s6 = s5.reset_index()
s6.to_parquet('D:/uber-taxi-demand/data/yellow_trip_data_2022.parquet', append=True, ignore_divisions=True)

In [26]:
s6.dask

HighLevelGraph with 12 layers.
<dask.highlevelgraph.HighLevelGraph object at 0x16cf2f91b10>
 0. read-parquet-1538b8ba5f0dfa34b8ce6ec0c6cc0001
 1. getitem-be2ee3e579450d728306f94ed6ea8cea
 2. gt-ed43cd7a6e66a412b7f4c429193e2a85
 3. loc-series-3537a53c9d6b408bbe8be61fb24ebf98
 4. set_index-8c511bf5763db725d35708eaef245416
 5. fillna-chunk-c5345fd5592e310103c28121b5f91614
 6. overlap-concat-c8498e85d0c9bca3abaf96e012474c9f
 7. overlap-ffill-69a8b85eb5682058f21461d453947b38
 8. repartition-merge-c4d90ec00e79615e13d07bc1adb586ca
 9. resample-b256c1a0b15183a5580f844b72e9f4f6
 10. optimeToFitMemory-082434232c98de518a765a29238ad730
 11. reset_index-a20d79def0187a2101cf64f6c5173721

In [6]:
files = [
    'D:/uber-taxi-demand/data/yellow_tripdata_2022-01.parquet',
    'D:/uber-taxi-demand/data/yellow_tripdata_2022-02.parquet',
    'D:/uber-taxi-demand/data/yellow_tripdata_2022-03.parquet',
    'D:/uber-taxi-demand/data/yellow_tripdata_2022-04.parquet',
    'D:/uber-taxi-demand/data/yellow_tripdata_2022-05.parquet',
    'D:/uber-taxi-demand/data/yellow_tripdata_2022-06.parquet',
    'D:/uber-taxi-demand/data/yellow_tripdata_2022-07.parquet',
    'D:/uber-taxi-demand/data/yellow_tripdata_2022-08.parquet',
    'D:/uber-taxi-demand/data/yellow_tripdata_2022-09.parquet',
    'D:/uber-taxi-demand/data/yellow_tripdata_2022-10.parquet',
    'D:/uber-taxi-demand/data/yellow_tripdata_2022-11.parquet',
    'D:/uber-taxi-demand/data/yellow_tripdata_2022-12.parquet',
]

In [7]:
optimized_df = pd.DataFrame(
    {
        'tpep_pickup_datetime': [pd.Timestamp(year=2022, month=1, day=1, hour=0, minute=0, second=0)],
        'passenger_count':[0,],
        'VendorID': [0,]
    }
)


for file in files:
    ddf = dd.read_parquet(file, npartitions=100)
    s1 = ddf.loc[
        ddf.tpep_pickup_datetime >= '2022-01-01 00:00:00', 
        ['tpep_pickup_datetime','passenger_count', 'VendorID']
    ]
    s2 = s1.set_index('tpep_pickup_datetime', sorted=True)
    s3 = s2.ffill()
    s4 = s3.resample('H').agg({'passenger_count': 'sum', 'VendorID': 'count'})
    s5 = s4.map_partitions(optimeToFitMemory)
    s6 = s5.reset_index()
    optimized_df = pd.concat([optimized_df, s6.compute()], axis=0, ignore_index=True)

In [8]:
optimized_df.shape

(21626, 3)

In [19]:
optimized_df.head()

,tpep_pickup_datetime,passenger_count,VendorID
0,2022-01-01 00:00:00,0,0
1,2022-01-01 00:00:00,6838,4261
2,2022-01-01 01:00:00,7738,4997
3,2022-01-01 02:00:00,5974,3901
4,2022-01-01 03:00:00,4107,2686


In [18]:
optimized_df.tail()

,tpep_pickup_datetime,passenger_count,VendorID
21621,2022-12-31 20:00:00,9419,6210
21622,2022-12-31 21:00:00,9081,5962
21623,2022-12-31 22:00:00,6732,4350
21624,2022-12-31 23:00:00,4873,3273
21625,2023-01-01 00:00:00,10,5


In [28]:
optimized_df.isnull().sum()

tpep_pickup_datetime    0
passenger_count         0
VendorID                0
dtype: int64

In [35]:
optimized_df.drop([0, 21625], inplace=True)

In [36]:
optimized_df.to_csv('D:/uber-taxi-demand/data/yellow_tripdata_2022.csv', index=False)

In [37]:
pd.read_csv('D:/uber-taxi-demand/data/yellow_tripdata_2022.csv', encoding='unicode_escape')

,tpep_pickup_datetime,passenger_count,VendorID
0,2022-01-01 00:00:00,6838,4261
1,2022-01-01 01:00:00,7738,4997
2,2022-01-01 02:00:00,5974,3901
3,2022-01-01 03:00:00,4107,2686
4,2022-01-01 04:00:00,2485,1651
...,...,...,...
21619,2022-12-31 19:00:00,8584,5730
21620,2022-12-31 20:00:00,9419,6210
21621,2022-12-31 21:00:00,9081,5962
21622,2022-12-31 22:00:00,6732,4350
